# SQL Analysis: Credit Risk Segmentation (SQLite)

## Goal
Use SQL to segment applicants and calculate default rates (Risk = 'bad') by:
- loan purpose
- duration buckets
- credit amount buckets

This mirrors how analysts explore risk drivers before modelling.


In [1]:
import pandas as pd
import sqlite3 
from pathlib import Path 

df=pd.read_csv("../data/german_credit_data.csv")
print(df.shape)
df.head()

(1000, 11)


,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


In [2]:
db_path = Path("../data/processed/credit_risk.db")

conn = sqlite3.connect(db_path)

# Write dataframe to a SQL table called credit_data
df.to_sql("credit_data", conn, if_exists="replace", index=False)

# Quick sanity check
pd.read_sql("SELECT COUNT(*) AS n_rows FROM credit_data;", conn)


,n_rows
0,1000


In [3]:
def q(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, conn)

q("PRAGMA table_info(credit_data);").head(20)


,cid,name,type,notnull,dflt_value,pk
0,0,Unnamed: 0,INTEGER,0,None,0
1,1,Age,INTEGER,0,None,0
2,2,Sex,TEXT,0,None,0
3,3,Job,INTEGER,0,None,0
4,4,Housing,TEXT,0,None,0
5,5,Saving accounts,TEXT,0,None,0
6,6,Checking account,TEXT,0,None,0
7,7,Credit amount,INTEGER,0,None,0
8,8,Duration,INTEGER,0,None,0
9,9,Purpose,TEXT,0,None,0


## Why SQLite?
SQLite lets me run SQL queries locally on the same cleaned dataset used for modelling.
This helps demonstrate practical SQL skills in a reproducible way.


## Default rate definition
The target column `Risk` is stored as text ('good'/'bad'), so I convert it to a numeric indicator in SQL:

CASE WHEN Risk = 'bad' THEN 1 ELSE 0 END

- SUM(...) gives the number of risky customers
- SUM(...)/COUNT(*) gives the default rate


In [4]:
q("""
SELECT
  Purpose AS purpose,
  COUNT(*) AS total_customers,
  SUM(CASE WHEN Risk = 'bad' THEN 1 ELSE 0 END) AS risky_customers,
  ROUND(AVG(CASE WHEN Risk = 'bad' THEN 1 ELSE 0 END), 3) AS default_rate
FROM credit_data
GROUP BY Purpose
ORDER BY default_rate DESC;
""")


,purpose,total_customers,risky_customers,default_rate
0,vacation/others,12,5,0.417
1,education,59,23,0.390
2,repairs,22,8,0.364
3,business,97,34,0.351
4,domestic appliances,12,4,0.333
5,furniture/equipment,181,58,0.320
6,car,337,106,0.315
7,radio/TV,280,62,0.221


In [5]:
q("""
SELECT
  Purpose AS purpose,
  COUNT(*) AS total_customers,
  SUM(CASE WHEN Risk = 'bad' THEN 1 ELSE 0 END) AS risky_customers,
  ROUND(
    1.0 * SUM(CASE WHEN Risk = 'bad' THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS default_rate
FROM credit_data
GROUP BY Purpose
HAVING COUNT(*) >= 50
ORDER BY default_rate DESC;
""")


,purpose,total_customers,risky_customers,default_rate
0,education,59,23,0.390
1,business,97,34,0.351
2,furniture/equipment,181,58,0.320
3,car,337,106,0.315
4,radio/TV,280,62,0.221


## Small sample warning
Some purposes have very low volume (e.g., ~12 rows). These can show extreme default rates due to small sample size.
I apply HAVING COUNT(*) >= 50 to focus on stable, decision-usable segments.


In [6]:
q("""
SELECT
  CASE
    WHEN Duration <= 12 THEN 'Short (<=12)'
    WHEN Duration <= 24 THEN 'Medium (13–24)'
    ELSE 'Long (25+)'
  END AS duration_bucket,
  COUNT(*) AS total_customers,
  SUM(CASE WHEN Risk = 'bad' THEN 1 ELSE 0 END) AS risky_customers,
  ROUND(
    1.0 * SUM(CASE WHEN Risk = 'bad' THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS default_rate
FROM credit_data
GROUP BY duration_bucket
HAVING COUNT(*) >= 50
ORDER BY default_rate DESC;
""")


,duration_bucket,total_customers,risky_customers,default_rate
0,Long (25+),230,102,0.443
1,Medium (13–24),411,122,0.297
2,Short (<=12),359,76,0.212


In [7]:
q("""
SELECT
  CASE
    WHEN "Credit amount" < 2000 THEN 'Low (<2k)'
    WHEN "Credit amount" < 5000 THEN 'Medium (2k–5k)'
    ELSE 'High (5k+)'
  END AS credit_bucket,
  COUNT(*) AS total_customers,
  SUM(CASE WHEN Risk = 'bad' THEN 1 ELSE 0 END) AS risky_customers,
  ROUND(
    1.0 * SUM(CASE WHEN Risk = 'bad' THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS default_rate
FROM credit_data
GROUP BY credit_bucket
HAVING COUNT(*) >= 50
ORDER BY default_rate DESC;
""")


,credit_bucket,total_customers,risky_customers,default_rate
0,High (5k+),188,78,0.415
1,Low (<2k),432,121,0.280
2,Medium (2k–5k),380,101,0.266


## Business interpretation (high level)
- Longer loan durations show higher default rates, which aligns with higher uncertainty over time.
- Higher credit amounts tend to be riskier due to larger exposure.
- If small loans show higher default rates than medium loans, this can reflect borrower profile differences or lighter screening for small loans.
